In [1]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai.foundation_models import Model
import time

In [2]:
from IPython.display import display, HTML
import pandas as pd

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>").replace("\\t","  ").replace("<td", "<td style='text-align:left'") ) )

# Document Generation

This notebook walks throught the process of using the meeting notes for a business need. In this use case we use them to modify a document. In any case we believe that the first step is to summarize the meeting notes with the final objective in mind.

In [3]:
# IBM Cloud Credentials
my_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : '< Your IBM Cloud API KEY >'
}      

## Summarization

This section generates a summary in form of a bullet list for each chunk of the transcript, a set of changes to be applied to the document. We are splitting and pasting the chunks manually. This could be done automatically in an MVP.

In [8]:
transcript_chunks = [
    '''Let's go with the abstract section. Let's see. Ok, is the objective still temporal visualization? Yeah, yeah, it's still still the same. That's that's okay. Ok, perfect. And what all the keywords? So, keywords, we have only these 4. I think I think we should add more specific visualizations into the keyword. So, let's add dendrogram, animated bubbles, animated lines, bars, and lines as well. Ok, sounds good. Let's go with that. now let's move on to the introduction section. Tell me a little bit about it. Ok, so let's go paragraph by paragraph. I think this first one, it's okay. You think these are the correct references that we are using? This date is correct. Ok, I think I think that that is good. 3. Revolution, okay. Yeah, I'll double-check that, but I think that that is fine as well. This Dupree, I think I think this second one, it's also ok. Let's jump into this last one. Uh, okay, there is something that we should be able to change here, right? 
    So, I think this wording is is a little bit off, where it says that the truth is that there is that it is very hard to compare players who played in different areas and different positions. I think we can do a little bit better here. So, I will change this last paragraph and these wording to say something around, you know, comparison between errors require a deeper analytics into the change of rules and players positions. Ok, I think I think that makes more sense. All right, yeah, ok. Let's let's go with that wording, making sure that this well now it looks more like a comparison between errors require a deeper analytics into the change of rules and players, yeah. Sounds good.'''
    '''Now, this the next is ok, background. Ok, background. Ok, this first paragraph looks ok to me. Yeah, this hasn't changed. Second paragraph, traditional positions, although well known, have never been strictly defined nor enforced. This is true, although not everyone might know the traditional positions. So, here, let's add the 5 traditional positions, Steve. So, that will be center, point guard, shooting guard, small forward, and forward. Yeah, that that will help a lot. And where should we add those? I think you just after it says, yeah, and background, second paragraph, just after it says, "m the well-known positions." Well-known, yeah. You can maybe at the parentheses or something that you can figure it out. Ok, perfect. So, we will add those 5 traditional positions there. Sounds good. And now, the next part will be, yes, approach. Ok, the approach section, yeah. This small section, yeah. This is this is ok. This is still the same. 
    Perfect. And now, Section datasets. Yeah, datasets, we did have a change here, a big change. We are not using anymore the basketball reference data set. So, what we should do to Steve is, in all the document, in all, in all the paper, we should change, sorry, we should change where it says, "back basketball reference" to "ESPN analytics." Ok, all right. So, good. Let's go with that change. Make sure to put the ESPN analytics instead. Yeah, yeah. Let's let's go with that. Other than that, let me, let me scroll down this first table. I think there's a change, since we change data sets. There is no offensive rebounds or defensive rebounds. So, we should delete these 2 rows and add one that says just total rebounds. Ok, perfect. Let's do that addition to that section, yeah. So, that table one, table 2, yeah. This one is still the same thing. Yeah, there is no change there.''',
    '''Perfect. Alright, now how about a section 3.3? 3.2, I think it's 3.2 first. Oh yeah, you're right. Tell me a little bit about that one. OK, 3.2 first, OK 3 point evolution per position. OK, in this section. I think I would add a paragraph. OK, let me let me see where. Where was that? OK, this chart is OK. Uh, this wording is is fine. This is what matters. Umm, this cluster names are still relevant. Here are here we are saying that we're gonna use these two algorithms. SARIMAX and AIC, and we should spend maybe a paragraph explaining what these two algorithms are. OK, so add a paragraph here at the end of the section and explaining what SARIMAX and AIC is OK. Perfect. We'll make that definition to the paragraph, so it helps out. Perfect. Ohh. and, let's move now to section. Now section 3.3. Sure. Yeah. OK. So this is the LeBron versus Michael umm inlet analysis. 
    OK, here we are good with the this assessment and I think there is something maybe we can change here Steve and that is. Umm, explain why the first 15 seasons here at the end of the first paragraph we could add a sentence that says something around that we only choose this 15 seasons because that's the amount of seasons that Michael Jordan played, OK. All right. OK, so in the first paragraph in section 3.3. We will explain that we use 15 season because that is ammount of season that Michael played. OK, perfect. Sounds good. Yeah, that is correct. OK. Results. Right moving section results perfect. OK. Yeah, Section 4 and four one. OK, Section 4.1, I think this is fine. This is just a link to the animation would change there. UM now to section 4.2 is there is other one. This is the Michael Jordan visualization again. We have the same. Two Visualization, I think I think I'm seeing here a lot of PER mentions, OK, yeah. 
    So let's let's modify this. So here where it says PER. Let's change it to player efficiency rating and then in parentheses PER OK. OK. We'll make that change sound good. Perfect.'''
]

In [43]:
def llama2_summary_prompt(transcript):
    return ( '[INST]<<SYS>>\nYou are a helpful, and honest assistant. You are asked to write a summary of the meeting transcripts including all the discussed changes to a document. Include the details necessary to make the discussed changes.\n\n'
            +'For example:\n\n'
            +'Text:\n'
            +'''In general, we are not using brand A, now we are using brand B. So, can we change all references from brand A to brand B? Yes. Now moving to the introduction, it is fine. We can leave it as it is. Now to the discussion section, we have changed the data upload process. We are no longer uploading the CSV files manually. Instead, the data is loaded to the database directly from the website. The cost section is all right, although we didn't have a workforce cost. What should we add for workforce? Let's put 15 USD per hour. Include the break down, 10 USD base salary, 2 USD for gas and 3 USD for food support. Tips not included. Ok. Moving to the constraints sections, we don't have any now. So we can delete the entire section? Yes.\n\n'''
            +'Output:\n'
            +'* Replace all references from brand A to brand B.\n'
            +'* Replace the references to manual uploading with data being directly loaded to the database from the website in the discussion section of the data upload process\n'
            +'* Add 15 USD per hour of workforce in the cost section with the following breakdown:\n'
            +'  + 10 USD as base salary.\n'
            +'  + 2 USD for gas.\n'
            +'  + 3 USD for food support.\n'
            +'  + Tips not included.\n'
            +'* Delete the constraints section.\n<</SYS>>\n\n'
            +f"Text:\n{transcript}\n\n"
            +'Output:[/INST]\n')

In [44]:
# This prompt template is an example for LLAMA3. You will need to tune it for your needs.
def llama3_summary_prompt(transcript):
    return ( '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are a helpful, and honest assistant. You are asked to write a summary of the meeting transcripts including all the discussed changes to a document. Include the details necessary to make the discussed changes.\n\n'
            +'For example:\n\n'
            +'Text:\n'
            +'''In general, we are not using brand A, now we are using brand B. So, can we change all references from brand A to brand B? Yes. Now moving to the introduction, it is fine. We can leave it as it is. Now to the discussion section, we have changed the data upload process. We are no longer uploading the CSV files manually. Instead, the data is loaded to the database directly from the website. The cost section is all right, although we didn't have a workforce cost. What should we add for workforce? Let's put 15 USD per hour. Include the break down, 10 USD base salary, 2 USD for gas and 3 USD for food support. Tips not included. Ok. Moving to the constraints sections, we don't have any now. So we can delete the entire section? Yes.\n\n'''
            +'Output:\n'
            +'* Replace all references from brand A to brand B.\n'
            +'* Replace the references to manual uploading with data being directly loaded to the database from the website in the discussion section of the data upload process\n'
            +'* Add 15 USD per hour of workforce in the cost section with the following breakdown:\n'
            +'  + 10 USD as base salary.\n'
            +'  + 2 USD for gas.\n'
            +'  + 3 USD for food support.\n'
            +'  + Tips not included.\n'
            +'* Delete the constraints section.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n'
            +f"Text:\n{transcript}\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
            +'Output:\n')

Now we use GenAI to summarize the chunks

In [15]:
model_id    = ModelTypes.LLAMA_3_70B_INSTRUCT
gen_parms   = None
project_id  = '< Your watsonx.ai project ID >'
space_id    = None
verify      = False

model = Model( model_id, my_credentials, gen_parms, project_id, space_id, verify )   

These are all the changes summarized:

In [16]:
parameters = {
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 800,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1
}

changes = []
for _chunk in transcript_chunks:
    prompt_txt = llama3_summary_prompt(_chunk)
    generated_response = model.generate_text( prompt_txt, parameters )
    changes.append(generated_response.strip())
    time.sleep(1)

print("".join(changes))

* In the abstract section:
  + Add the following keywords: dendrogram, animated bubbles, animated lines, bars, and lines.
* In the introduction section:
  + Revise the last paragraph to read: "Comparison between errors require a deeper analytics into the change of rules and players positions."
* In the background section:
  + Add the 5 traditional positions (center, point guard, shooting guard, small forward, and forward) in parentheses after the phrase "well-known positions" in the second paragraph.
* In the datasets section:
  + Replace all references to "basketball reference" with "ESPN analytics" throughout the document.
  + In table 1, delete the rows for "offensive rebounds" and "defensive rebounds" and add a new row for "total rebounds".* Add a paragraph to the end of section 3.2 explaining what SARIMAX and AIC algorithms are.
* In section 3.3, add a sentence to the first paragraph explaining that the first 15 seasons were chosen because that is the amount of seasons Michael Jor

These are the set of changes for each chunk of the transcript. The goal is to validate the accuracy of the changes (Corrects, Inclusions, Omissions, etc)

In [17]:
df = pd.DataFrame(zip(transcript_chunks, changes), columns=["Transcript", "Changes"])
pretty_print(df)

,Transcript,Changes
0,"Let's go with the abstract section. Let's see. Ok, is the objective still temporal visualization? Yeah, yeah, it's still still the same. That's that's okay. Ok, perfect. And what all the keywords? So, keywords, we have only these 4. I think I think we should add more specific visualizations into the keyword. So, let's add dendrogram, animated bubbles, animated lines, bars, and lines as well. Ok, sounds good. Let's go with that. now let's move on to the introduction section. Tell me a little bit about it. Ok, so let's go paragraph by paragraph. I think this first one, it's okay. You think these are the correct references that we are using? This date is correct. Ok, I think I think that that is good. 3. Revolution, okay. Yeah, I'll double-check that, but I think that that is fine as well. This Dupree, I think I think this second one, it's also ok. Let's jump into this last one. Uh, okay, there is something that we should be able to change here, right? So, I think this wording is is a little bit off, where it says that the truth is that there is that it is very hard to compare players who played in different areas and different positions. I think we can do a little bit better here. So, I will change this last paragraph and these wording to say something around, you know, comparison between errors require a deeper analytics into the change of rules and players positions. Ok, I think I think that makes more sense. All right, yeah, ok. Let's let's go with that wording, making sure that this well now it looks more like a comparison between errors require a deeper analytics into the change of rules and players, yeah. Sounds good.Now, this the next is ok, background. Ok, background. Ok, this first paragraph looks ok to me. Yeah, this hasn't changed. Second paragraph, traditional positions, although well known, have never been strictly defined nor enforced. This is true, although not everyone might know the traditional positions. So, here, let's add the 5 traditional positions, Steve. So, that will be center, point guard, shooting guard, small forward, and forward. Yeah, that that will help a lot. And where should we add those? I think you just after it says, yeah, and background, second paragraph, just after it says, ""m the well-known positions."" Well-known, yeah. You can maybe at the parentheses or something that you can figure it out. Ok, perfect. So, we will add those 5 traditional positions there. Sounds good. And now, the next part will be, yes, approach. Ok, the approach section, yeah. This small section, yeah. This is this is ok. This is still the same. Perfect. And now, Section datasets. Yeah, datasets, we did have a change here, a big change. We are not using anymore the basketball reference data set. So, what we should do to Steve is, in all the document, in all, in all the paper, we should change, sorry, we should change where it says, ""back basketball reference"" to ""ESPN analytics."" Ok, all right. So, good. Let's go with that change. Make sure to put the ESPN analytics instead. Yeah, yeah. Let's let's go with that. Other than that, let me, let me scroll down this first table. I think there's a change, since we change data sets. There is no offensive rebounds or defensive rebounds. So, we should delete these 2 rows and add one that says just total rebounds. Ok, perfect. Let's do that addition to that section, yeah. So, that table one, table 2, yeah. This one is still the same thing. Yeah, there is no change there.","* In the abstract section: + Add the following keywords: dendrogram, animated bubbles, animated lines, bars, and lines.* In the introduction section: + Revise the last paragraph to read: ""Comparison between errors require a deeper analytics into the change of rules and players positions.""* In the background section: + Add the 5 traditional positions (center, point guard, shooting guard, small forward, and forward) in parentheses after the phrase ""well-known positions"" in the secon

## Classify Changes
In this section we classify the changes to be executed in the proper order.

In [18]:
def mixtral_classification_prompt(action):
    return ( '[INST]Classify this action into one of four classes: Delete, Add, Replace, No Change.\n\n'
            +'Class name: Delete\n'
            +'Description: The action asks to delete or remove an entire section or a portion.\n\n'
            +'Class name: Add\n'
            +'Description: The action asks to add a new section or paragraph.\n\n'
            +'Class name: Replace\n'
            +'Description: The action asks to replace one reference with another.\n\n'
            +'Class name: No Change\n'
            +'Description: The action states that no changes are to be made.\n\n'
            +f"Action:{action}\n"
            +'Just generate the class name without explanations:\n'
            +'[/INST]')

In [19]:
all_changes = []
for _changes in changes:
    for _change in _changes.split('*'):
        if len(_change)>5:
            all_changes.append(_change)

print(f"There are {len(all_changes)} changes")


There are 7 changes


Now we use GenAI to classify the changes into: Delete, Add, Replace, or No Change

In [20]:
model_id    = ModelTypes.MIXTRAL_8X7B_INSTRUCT_V01
gen_parms   = None
project_id  = '1dbd8eb5-588c-4f7e-98a8-6c65a179fc9d'
space_id    = None
verify      = False

model_mix = Model( model_id, my_credentials, gen_parms, project_id, space_id, verify )   

In [21]:
parameters_mix = {
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.MAX_NEW_TOKENS: 5,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1
}


_delete = []
_add = []
_replace = []
_nochange = []
for _change in all_changes:
    prompt_txt = mixtral_classification_prompt(_change)
    generated_response = model_mix.generate_text( prompt_txt, parameters_mix ).lower().strip()
    if 'delete' in generated_response:
        _delete.append(_change)
    elif 'add' in generated_response:
        _add.append(_change)
    elif 'replace' in generated_response:
        _replace.append(_change)
    elif 'no change' in generated_response:
        _nochange.append(_change)
    else:
        print('Misclassifications')
        print('Action: '+_change)
        print(generated_response)
        _replace.append(_change) # Default to replace
    time.sleep(1)

These are the classified changes.

In [22]:
_nochange

[]

In [23]:
_delete

[' In the datasets section:\n  + Replace all references to "basketball reference" with "ESPN analytics" throughout the document.\n  + In table 1, delete the rows for "offensive rebounds" and "defensive rebounds" and add a new row for "total rebounds".']

In [24]:
_add

[' In the abstract section:\n  + Add the following keywords: dendrogram, animated bubbles, animated lines, bars, and lines.\n',
 ' In the background section:\n  + Add the 5 traditional positions (center, point guard, shooting guard, small forward, and forward) in parentheses after the phrase "well-known positions" in the second paragraph.\n',
 ' Add a paragraph to the end of section 3.2 explaining what SARIMAX and AIC algorithms are.\n',
 ' In section 3.3, add a sentence to the first paragraph explaining that the first 15 seasons were chosen because that is the amount of seasons Michael Jordan played.\n']

In [25]:
_replace

[' In the introduction section:\n  + Revise the last paragraph to read: "Comparison between errors require a deeper analytics into the change of rules and players positions."\n',
 ' In section 4.2, replace all instances of "PER" with "player efficiency rating (PER)".']

## Generation by Sets

The objective of this experiment is to concatenate the output of one set of changes into the generation of the next set of changes.
The goal is to have at the end a document with all the changes applied and to be able to measure its performance.

In [41]:
def mix_gen_prompt_1(changes, text):
    return ( '[INST] <<SYS>>\nYou are a helpful, and honest assistant. Modify the text according only to the listed changes.\n'
            +f'Changes:\n{changes}\n'
            +'<</SYS>>\n\n'
            +f"Text:\n\n{text}\n[/INST]\nHere's the modified text with the requested changes:\n" )

In [42]:
def llama3_gen_prompt(changes, text):
    return ( '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are an honest assistant. Modify the text according only to the listed changes.\n'
            +f'Changes:\n{changes}\n'
            +'<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n'
            +f"Text:\n\n{text}\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\nHere's the modified text with the requested changes:\n" )

Now we apply the changes in order, first all the delete, then replace and finally the addition.

In [38]:
parameters = {
    GenParams.MIN_NEW_TOKENS: 500,
    GenParams.MAX_NEW_TOKENS: 4000,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1
}

prev_doc_text = ''

with open('document.txt') as f:
    # Read the contents of the file into a variable
    prev_doc_text = f.read()


doc_text_sets = prev_doc_text
doc_changed_sets = []

_delete_changes = '*' + "*".join(_delete)
_replace_changes = '*' + "*".join(_replace)
_add_changes = '*' + "*".join(_add)

prompt_txt_delete = llama3_gen_prompt(_delete_changes, doc_text_sets)
generated_response = model.generate_text( prompt_txt_delete, parameters )
doc_text_sets = generated_response.strip()
doc_changed_sets.append(doc_text_sets)
time.sleep(1)

prompt_txt_replace = llama3_gen_prompt(_replace_changes, doc_text_sets)
parameters[GenParams.MIN_NEW_TOKENS] = int(model.tokenize(prompt=doc_text_sets, return_tokens=True)['result']['token_count'])
generated_response = model.generate_text( prompt_txt_replace, parameters )
doc_text_sets = generated_response.strip()
doc_changed_sets.append(doc_text_sets)
time.sleep(1)

prompt_txt_add = llama3_gen_prompt(_add_changes, doc_text_sets)
parameters[GenParams.MIN_NEW_TOKENS] = int(model.tokenize(prompt=doc_text_sets, return_tokens=True)['result']['token_count'])
generated_response = model.generate_text( prompt_txt_add, parameters )
doc_text_sets = generated_response.strip()
doc_changed_sets.append(doc_text_sets)
time.sleep(1)    


These are the results after applying each set of changes. Remember that these changes are cumulative, you should see the changes hold at each step.

In [39]:
df = pd.DataFrame(zip([_delete_changes,_replace_changes,_add_changes], doc_changed_sets), columns=['Changes', 'Modified'])
pretty_print(df)

This is the comparision between the orignal document and the final modified text after applying all the changes.

In [40]:
df = pd.DataFrame([(prev_doc_text, doc_text_sets)], columns=['Original', 'Modified'])
pretty_print(df)